In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import dask

import glob
from cdo import *
import os
import sys
from datetime import datetime

import cftime

sys.path.append('/glade/u/home/currierw/cmip_ingest/scripts')
from download import get_dataset
sys.path.append('/glade/u/home/currierw/cmip_work')
import cmipFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
cdo = Cdo()

# Parameters
# Slice/Subset data down further based on latitutde (-90 -- +90), longitude (0 -- 360)
lat_bnds, lon_bnds = [22, 58], [230, 265]
start_date = "19500101"
end_date   = "20060101"
start = ['1950-01-01 06:00:00','1960-01-01 06:00:00','1970-01-01 06:00:00','1980-01-01 06:00:00','1990-01-01 06:00:00','2000-01-01 06:00:00']
end   = ['1960-01-01 00:00:00','1970-01-01 00:00:00','1980-01-01 00:00:00','1990-01-01 00:00:00','2000-01-01 00:00:00','2006-01-01 00:00:00']
Institute   = 'BCC'
Model       = 'bcc-csm1-1'  # ACCESS1-0
time_period = 'historical'  # historical, rcp45, rcp85
ensemble    = 'r1i1p1'      # r1i1p1
version     = 'v1'
chyColl     = True         # Data exist in cheyenne collection already - see list

In [ ]:
chyModelList=['BCC/bcc-csm1-1','BCC/bcc-csm1-1-m','BNU/BNU-ESM','CCCma/CanESM2','CNRM-CERFACS/CNRM-CM5','CSIRO-BOM/ACCESS1-0','CSIRO-BOM/ACCESS1-3', \
              'CSIRO-QCCCE/CSIRO-Mk3-6-0','IPSL/IPSL-CM5A-LR','IPSL/IPSL-CM5A-MR','IPSL/IPSL-CM5B-LR', \
              'LASG-CESS/FGOALS-g2','MIROC/MIROC5','MIROC/MIROC-ESM-CHEM','MIROC/MIROC-ESM','MOHC/HadGEM2-ES','MRI/MRI-CGCM3', \
              'NCC/NorESM1-M','NOAA-GFDL/GFDL-CM3','NOAA-GFDL/GFDL-ESM2G']

fName="/glade/u/home/currierw/cmip_work/CMIP5historical_r1i1p1.txt"




# ['ACCESS1-0:         on cheyenne',
#  'ACCESS1-3:         on cheyenne',
#  'bcc-csm1-1:        on cheyenne',
#  'bcc-csm1-1-m:      on cheyenne',
#  'BNU-ESM:           on cheyenne',
#  'CanESM2:           on cheyenne',
#  'CCSM4:         not on cheyenne', - on /colllection/cdg/data/cmip5... r6i1pi1 though
#  'CESM1-BGC:     not on cheyenne',
#  'CESM1-CAM5:    not on cheyenne',
#  'CMCC-CM:       not on cheyenne',
#  'CMCC-CMS:      not on cheyenne',
#  'CNRM-CM5:          on cheyenne',
#  'CSIRO-Mk3-6-0:     on cheyenne',
#  'EC-EARTH:      not on cheyenne',
#  'FGOALS-g2:         on cheyenne',
#  'FIO-ESM:       not on cheyenne',
#  'GFDL-CM3:          on cheyenne',
#  'GFDL-ESM2G:        on cheyenne',
#  'GFDL-ESM2M:        on cheyenne',
#  'GISS-E2-H-cc:  not on cheyenne',
#  'GISS-E2-H:     not on cheyenne',
#  'GISS-E2-R-cc:  not on cheyenne',
#  'GISS-E2-R:     not on cheyenne',
#  'GISS-E2-R:     not on cheyenne', ?
#  'HadGEM2-AO:    not on cheyenne',
#  'HadGEM2-CC:    not on cheyenne',
#  'HadGEM2-ES:        on cheyenne',
#  'HadGEM2-ES:        on cheyenne',
#  'inmcm4:        not on cheyenne',
#  'IPSL-CM5A-LR:      on cheyenne',
#  'IPSL-CM5A-MR:      on cheyenne',
#  'IPSL-CM5B-LR:      on cheyenne',
#  'MIROC-ESM:         on cheyenne',
#  'MIROC-ESM-CHEM:    on cheyenne',
#  'MIROC5:            on cheyenne',
#  'MPI-ESM-LR:    not on cheyenne',
#  'MPI-ESM-MR:    not on cheyenne',
#  'MRI-CGCM3:         on cheyenne',
#  'NorESM1-M:         on cheyenne']

#  Models on Cheyenne            Staggered?   dy                  dx
#  ---------------------------------------------------------------------------------
#  'BCC/bcc-csm1-1:            not staggered  2.7672718509152645  2.8125',
#  'BCC/bcc-csm1-1-m:          not staggered  1.1120905585095784  1.125',
#  'BNU/BNU-ESM:               not staggered  2.76727294921875    2.8125',
#  'CCCma/CanESM2:             not staggered  2.7672718509152645  2.8125',
#  'CNRM-CERFACS/CNRM-CM5:     not staggered  1.3890304565429688  1.40625',
#  'CSIRO-BOM/ACCESS1-0:           staggered  1.25                1.875',
#  'CSIRO-BOM/ACCESS1-3:           staggered  1.25                1.875',
#  'CSIRO-QCCCE/CSIRO-Mk3-6-0: not staggered  1.8496322631835938  1.875',
#  'IPSL/IPSL-CM5A-LR:         not staggered  1.8947372436523438  3.75',
#  'IPSL/IPSL-CM5A-MR:         not staggered  1.267608642578125   2.5',
#  'IPSL/IPSL-CM5B-LR:         not staggered  1.8947372436523438  3.75',
#  'LASG-CESS/FGOALS-g2:       not staggered  6.131454469999994   2.8125',
#  'MIROC/MIROC5:              not staggered  1.3890301392687405  1.40625',
#  'MIROC/MIROC-ESM-CHEM:      not staggered  2.7672718509152645  2.8125',
#  'MIROC/MIROC-ESM:           not staggered  2.7672718509152645  2.8125',
#  'MOHC/HadGEM2-ES:               staggered  1.25                1.875',
#  'MRI/MRI-CGCM3:             not staggered  1.112089999999995   1.125',
#  'NCC/NorESM1-M:             not staggered  1.8947368421052602  2.5',
#  'NOAA-GFDL/GFDL-CM3:        not staggered  2.0                 2.4999999999999996',
#  'NOAA-GFDL/GFDL-ESM2G:      not staggered  1.516853932584283   2.4999999999999996'

stagLs=[]
dxLs=[]
dyLs=[]
for i in range(0,len(chyModelList)):

    fDirs,varDir,taFiles = cmipFunctions.searchChyColl(fName,chyModelList[i],'ta')
    dsTa  = xr.open_dataset(taFiles[0])
    
    fDirs,varDir,uaFiles = cmipFunctions.searchChyColl(fName,chyModelList[i],'ua')
    dsUa  = xr.open_dataset(uaFiles[0])

    fDirs,varDir,vaFiles = cmipFunctions.searchChyColl(fName,chyModelList[i],'va')
    dsVa  = xr.open_dataset(vaFiles[0])
    
    diffLat = np.abs(dsVa.lat[0]-dsUa.lat[0])
    diffLon = np.abs(dsVa.lon[0]-dsUa.lon[0])
    
    if diffLat>0 or diffLon>0:
        stagLs.append(chyModelList[i]+': staggered '+str(np.abs((dsTa.lat[1]-dsTa.lat[0]).values))+' '+str(np.abs((dsTa.lon[1]-dsTa.lon[0]).values)))
    else:
        stagLs.append(chyModelList[i]+': not staggered '+str(np.abs((dsTa.lat[1]-dsTa.lat[0]).values))+' '+str(np.abs((dsTa.lon[1]-dsTa.lon[0]).values)))
        

In [2]:
GCMList=['ACCESS1-0','ACCESS1-3','bcc-csm1-1','bcc-csm1-1-m','BNU-ESM','CanESM2','CCSM4','CESM1-BGC', \
    'CESM1-CAM5','CMCC-CM','CMCC-CMS','CNRM-CM5','CSIRO-Mk3-6-0','EC-EARTH','FGOALS-g2','FIO-ESM', \
    'GFDL-CM3','GFDL-ESM2G','GFDL-ESM2M','GISS-E2-H-cc','GISS-E2-H','GISS-E2-R-cc','GISS-E2-R', \
    'GISS-E2-R','HadGEM2-AO','HadGEM2-CC','HadGEM2-ES','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','IPSL-CM5A-LR', \
    'IPSL-CM5A-LR','IPSL-CM5A-LR','IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC-ESM','MIROC-ESM-CHEM','MIROC5', \
    'MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M']
fName="/glade/u/home/currierw/cmip_work/CMIP5historical_r1i1p1.txt"

GCMChy=[]
for i in range(0,len(GCMList)):
    fDirs,varDir,taFiles = cmipFunctions.searchChyColl(fName,GCMList[i],'ta')
    if len(varDir)==0:
        GCMChy.append(GCMList[i]+': not on cheyenne')
    else:
        GCMChy.append(GCMList[i]+': on cheyenne')


In [3]:
GCMChy

['ACCESS1-0: on cheyenne',
 'ACCESS1-3: on cheyenne',
 'bcc-csm1-1: on cheyenne',
 'bcc-csm1-1-m: on cheyenne',
 'BNU-ESM: on cheyenne',
 'CanESM2: on cheyenne',
 'CCSM4: not on cheyenne',
 'CESM1-BGC: not on cheyenne',
 'CESM1-CAM5: not on cheyenne',
 'CMCC-CM: not on cheyenne',
 'CMCC-CMS: not on cheyenne',
 'CNRM-CM5: on cheyenne',
 'CSIRO-Mk3-6-0: on cheyenne',
 'EC-EARTH: not on cheyenne',
 'FGOALS-g2: on cheyenne',
 'FIO-ESM: not on cheyenne',
 'GFDL-CM3: on cheyenne',
 'GFDL-ESM2G: on cheyenne',
 'GFDL-ESM2M: on cheyenne',
 'GISS-E2-H-cc: not on cheyenne',
 'GISS-E2-H: not on cheyenne',
 'GISS-E2-R-cc: not on cheyenne',
 'GISS-E2-R: not on cheyenne',
 'GISS-E2-R: not on cheyenne',
 'HadGEM2-AO: not on cheyenne',
 'HadGEM2-CC: not on cheyenne',
 'HadGEM2-ES: on cheyenne',
 'HadGEM2-ES: on cheyenne',
 'inmcm4: not on cheyenne',
 'IPSL-CM5A-LR: on cheyenne',
 'IPSL-CM5A-LR: on cheyenne',
 'IPSL-CM5A-LR: on cheyenne',
 'IPSL-CM5A-LR: on cheyenne',
 'IPSL-CM5A-MR: on cheyenne',
 'IPS